In [1]:
import requests
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import zipfile
from tqdm import tqdm
import base64
import pathlib

In [2]:
load_dotenv()

True

In [41]:
tables = ['users', 'datacenters', 'connections']
schemas = []
for table in tables:
    response = requests.post('https://centrala.ag3nts.org/apidb', json={
        "task": "database",
        "apikey": os.getenv('AI_DEVS_3_API_KEY'),
        "query": f"show create table {table}"
    })
    schemas.append(response.json()['reply'][0]['Create Table'].replace('\n', ''))

In [42]:
schemas

["CREATE TABLE `users` (  `id` int(11) NOT NULL AUTO_INCREMENT,  `username` varchar(20) DEFAULT NULL,  `access_level` varchar(20) DEFAULT 'user',  `is_active` int(11) DEFAULT 1,  `lastlog` date DEFAULT NULL,  PRIMARY KEY (`id`)) ENGINE=InnoDB AUTO_INCREMENT=99 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci",
 'CREATE TABLE `datacenters` (  `dc_id` int(11) DEFAULT NULL,  `location` varchar(30) NOT NULL,  `manager` int(11) NOT NULL DEFAULT 31,  `is_active` int(11) DEFAULT 0) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci',
 'CREATE TABLE `connections` (  `user1_id` int(11) NOT NULL,  `user2_id` int(11) NOT NULL,  PRIMARY KEY (`user1_id`,`user2_id`)) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci']

In [43]:
client = OpenAI()

In [55]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f'Oto schematy tabel w mojej bazie danych: {schemas}'},
        {"role": "user", "content": "Przygotuj zapytanie sql odpowiadające na pytanie: \
            które aktywne datacenter (DC_ID) są zarządzane przez pracowników, którzy są na urlopie (is_active=0). \
                Napisz wyłącznie gotowe do użycia query bez znaków nowej linii czy niepotrzebnych znaków"}
    ]
)
answer = response.choices[0].message.content.replace('\n', '').replace('`', '').replace('sql', '')

In [56]:
answer

'SELECT datacenters.dc_id FROM datacenters JOIN users ON datacenters.manager = users.id WHERE datacenters.is_active = 1 AND users.is_active = 0'

In [57]:
response = requests.post('https://centrala.ag3nts.org/apidb', json={
    "task": "database",
    "apikey": os.getenv('AI_DEVS_3_API_KEY'),
    "query": answer
})

In [58]:
response.json()

{'reply': [{'dc_id': '4278'}, {'dc_id': '9294'}], 'error': 'OK'}

In [59]:
solution_response = {
   "task": "database",
   "apikey": os.getenv('AI_DEVS_3_API_KEY'),
   "answer": ['4278', '9294']
}

In [61]:
response = requests.post('https://centrala.ag3nts.org/report', json=solution_response)

In [62]:
response

<Response [200]>

In [63]:
response.json()

{'code': 0, 'message': '{{FLG:KNOWLEDGE}}'}